# ARTIGO 03 — Exógeno + Latência (Kairos Alloy)

Referência: `research/ARTIGO 3 - Informação Exógena Observável no Tempo para DRL em Criptomoedas - Latência e Valor Marginal.md`.

Objetivo deste notebook: comparar variantes do mesmo protocolo (ex.: `T` vs `T+X`) sob cenários de latência/placebos,
lendo apenas artefatos padronizados em `runs/<run_id>/`.


In [ ]:
RUN_IDS = []  # e.g. ["run_a", "run_b"] ; empty => latest only
RUNS_DIR = "runs"
EXPORT_NAME = "paper"


In [ ]:
from pathlib import Path
import sys
import json

import pandas as pd

cwd = Path.cwd().resolve()
repo_root = cwd if (cwd / "crates").exists() else cwd.parent
notebooks_dir = repo_root / "notebooks"
sys.path.insert(0, str(notebooks_dir / "_lib"))

from runs import load_run, list_runs, latest_run
from dataframes import read_trades_csv, read_equity_csv
from plots import plot_equity_curve

runs_dir = (repo_root / RUNS_DIR).resolve()
if not RUN_IDS:
    rd = latest_run(runs_dir)
    if rd is None:
        raise RuntimeError(f"no runs found under {runs_dir}")
    RUN_IDS = [rd.name]

loaded = [load_run(runs_dir / rid) for rid in RUN_IDS]
[(r.run_id, bool(r.summary), bool(r.trades_csv), bool(r.equity_csv)) for r in loaded]

## Quick compare (summary)

Dica: quando vocês adicionarem `manifest.json` por run (protocol_id/variant_id/latency/etc.),
este notebook pode filtrar automaticamente por tratamento.


In [ ]:
rows = []
for r in loaded:
    s = r.summary or {}
    rows.append({
        "run_id": r.run_id,
        "bars_processed": s.get("bars_processed"),
        "trades": s.get("trades"),
        "net_profit": s.get("net_profit"),
        "sharpe": s.get("sharpe"),
        "max_drawdown": s.get("max_drawdown"),
    })
pd.DataFrame(rows)

## Equity curves (per run)


In [ ]:
for r in loaded:
    if r.equity_csv is None:
        continue
    eq = read_equity_csv(r.equity_csv)
    _, figures_dir = r.ensure_paper_dirs()
    out = figures_dir / f"equity_{r.run_id}.png"
    plot_equity_curve(eq, title=f"Equity — {r.run_id}", save_path=out)
    print(out)

## Trades + reason sanity

Para ARTIGO 03, `reason` vira essencial para auditar decisões e detectar placebos/leak.


In [ ]:
for r in loaded:
    if r.trades_csv is None:
        continue
    tr = read_trades_csv(r.trades_csv)
    top = tr["reason"].fillna("").value_counts().head(10)
    print("\n===", r.run_id)
    print(top.to_string())

## TODO (ARTIGO 03)

- Agrupar runs por (T, T+X, placebos) × latência (L0/L1/L2).
- Exportar tabelas do paper: efeito incremental Δ e teste contra placebos.
- Adicionar checagens de observabilidade temporal (quando o dataset/manifest existir).
